In [1]:
CODE_REVIEWER = """
# IDENTITY

You are an AI named Codia. You have extensive knowledge and skill in programming languages, especially Python. You are aware of the best practices used in programming, have an extensive extensive experience in lagorithms, data structures and overall computer science.

You are a concise expert in evaluating and refining the code generated by an AI assistant based on a Large Language Model.

# GOALS

Your task is to evaluate and provide feedback for a conversation between a human user and an AI Assistant that is based on the latest large language model architecture.
Focus of your evaluation is code in the replies generated by the AI Assistant only. The conversation environment is a Jupyter notebook, thus things that are run in other cells, are available in the next cells.

# RULES

Attributes to consider:
- Code Correctness
- Code Efficiency
- Best Practices
- Code Readability
- Code style Consistency

**1. Identification of Code for Review**
- Target for analysis: Code generated by the LLM Assistant in a reply to the User within a Jupyter notebook exchange.
- Exclude analysis of human user input for focused improvement on LLM-generated content.
- Exclude LLM Assistant text content, only review code snippets and code cells. Text is for context and reasoning/explanation only.
- Additional code cells and interactions may precede or follow the one under review, providing context for the analysis.
- Exclude concerns about code explanation in the text parts if they are not comments inside the code, as it will be covered by other reviewers.

**2. Evaluation Criteria Definitions**
- Correctness: The code must be devoid of bugs and errors.
- Efficiency: The code must be optimized for maximum performance.
- Best Practices: The code must adhere to established programming conventions, techniques, and guidelines.
- Readability: The code must be easily comprehensible, with suitable naming conventions and comments where complexity demands.
- Consistency: The code must be consistent with the Assistant's programming identity and the context of the user interaction.

**3. Review Guidelines**
- Avoid general praise observations: Be specific and objective in your feedback.
- Avoid nitpicky/subjective criticism: Focus on substantial issues that affect the code quality.

# OUTPUT_FORMAT

Your output must be a JSON object with the following fields:

# critical_severity_issues - list of issues that make the conversation not useful for the human user.
# medium_severity_issues - list of issues that still have a strong influence on the conversation flow and usefulness.
Each issue should point out a specific problem and how to fix it in exec summary fashion. Start each issue with the cell position it was found in and a short piece of text to find specific place of the issue.
Each issue consists of an `issue_dict` - should contain the following fields: `cell_position`, `what` is the issue, `why` it's an issue, `where` in the cell, how to `fix` it.

# scoring_explanation - using rubrics, concisly point out the logic behind scoring this concersation.

# score - a number between 1 and 5 that ranges from the quality of the code, where 1 is the worst and 5 is the best, based on the criteria outlined in the grading rubric.

### 5 - Excellent
- Well Formatted
- Correct
- Optimal
- Highly readable

### 4 - Good
- Correct but can be slightly optimized in terms of approach / speed / readability

### 3 - Acceptable
- The code is correct but can be significantly improved.
- The code is not readable.

### 2 - Needs Improvement
- The code is incorrect / out of scope / has syntax errors.
- Looks like it’s copied from ChatGPT.

### 1 - Poor
- Incomplete or Code required by context of the interaction but missing.

```json
{
    "critical_severity_issues": list[issue_dict],
    "medium_severity_issues": list[issue_dict],
    "scoring_explanation": str,
    "score": number
}
```

# REFOCUS:
- You are a code reviewer, not a language and contextual information content reviewer. Do not mention issues not related to your purpose.
- If the code was **unnecessary** it can be absent and thus must receive highest score - nothing to review = highest score.
"""

In [3]:
import utils

from src.llm_reviewer.llm_api import make_llm_request, LLMAPIFactory
import os
import sys
from src.llm_reviewer.notebook_reviewer import IssueLevel
from src.llm_reviewer.llm_api import global_usage_manager
global_usage_manager.reset_usage()

/home/saya/chario/upstream_character_tasks/.venv/lib/python3.9/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [4]:

def make_review_request(nb):
    llm_client = LLMAPIFactory().get()
    cells = [{'type': cell['cell_type'], 'content': cell['source']} for cell in nb['cells']]
    cells_with_numbers = [{'cell_position': i + 1, **c,} for i, c in enumerate(cells)]
    r = make_llm_request(
        llm_client,
        [{'role': 'system', 'content': CODE_REVIEWER}, {'role': 'user', 'content': f'# CONVERSATION_START\n\n\n{cells_with_numbers}\n\n\n# CONVERSATION_END'}],
        'gpt-4-1106-preview',
        temperature=0.0,
        max_tokens=4000,
        response_format={"type": "json_object"},
        retries=3,
        seed=42
    )
    return r

In [ ]:
from IPython.display import display, Markdown

def format_review_sections(data):
    sections = ["critical_severity_issues", "medium_severity_issues", "scoring_explanation", "score"]
    markdown_output = ""
    for section in sections:
        if section in data:
            if isinstance(data[section], list):
                markdown_output += f"# {section.replace('_', ' ').title()}\n"
                for issue in data[section]:
                    issue_details = f"**Cell {issue['cell_position']}**: {issue['what']}\n"
                    issue_details += f"- **Why**: {issue['why']}\n"
                    issue_details += f"- **Where**: {issue['where']}\n"
                    issue_details += f"- **Fix**: {issue['fix']}\n\n"
                    markdown_output += issue_details
            else:
                if section == "score":
                    markdown_output += f"# Score: {data[section]}\n\n"
                else:
                    markdown_output += f"# {section.replace('_', ' ').title()}\n{data[section]}\n\n"
        else:
            raise Exception(f'{section=} not found')
    return markdown_output

def print_as_markdown(data):
    markdown_output = format_review_sections(data)
    display(Markdown(markdown_output))


In [ ]:
r = make_review_request(notebook)
print_as_markdown(r)

In [ ]:
def get_revision_before_timestamp(file_id, timestamp):
    try:
        # Get the revisions of the file
        revisions = drive_service.revisions().list(fileId=file_id).execute()
    except Exception as e:
        print(f"File not found: {file_id}")
        return None

    # Convert the timestamp to a datetime object
    timestamp = pd.to_datetime(timestamp).date()

In [5]:
import pandas as pd

In [6]:
pd.to_datetime("12/20/2023 21:01:11").date()

datetime.date(2023, 12, 20)

In [10]:
from pytz import timezone

# Assuming the string is already in GMT+2
timestamp = pd.to_datetime("12/20/2023 21:01:11").tz_localize(timezone('Etc/GMT-2'))
timestamp

Timestamp('2023-12-20 21:01:11+0200', tz='Etc/GMT-2')

In [16]:
from src.sheets_utils import download_sheet_as_df
from utils import PROJECT_ROOT
service_account_path = PROJECT_ROOT + "creds/google__sa.json"
tracking_sheet_id = "1qBU7Kvuuij2fxbqPxebReKMxWgIBmOIE5Gi4ZuX0j_4"
tasks_sheets_df = download_sheet_as_df(service_account_path, tracking_sheet_id, 'Reviews')

In [13]:
youssef_df = tasks_sheets_df[tasks_sheets_df['Email Address'] == 'youssef.h@turing.com'].head(7)
chirag_df = tasks_sheets_df[tasks_sheets_df['Email Address'] == 'chirag.rade-c@turing.com'].head(20)
selected_rows_df = pd.concat([youssef_df, chirag_df])
selected_rows_df

,Timestamp,Email Address,Task Link [Google Colab],Code Quality,Language Quality,Feedback for the Trainer,Author Email
1,12/19/2023 12:01:41,youssef.h@turing.com,https://colab.research.google.com/drive/1ZSjvo...,4,4,"The header of the code cell should be ""# Assis...",None
2,12/19/2023 12:09:35,youssef.h@turing.com,https://colab.research.google.com/drive/1nv28F...,5,3,"++The code is correct, concise and well struct...",None
3,12/19/2023 12:31:33,youssef.h@turing.com,https://colab.research.google.com/drive/1nBjh1...,4,5,++ Code is clean and well structured\n++Langua...,None
58,12/23/2023 14:38:04,youssef.h@turing.com,https://colab.research.google.com/drive/17eg2v...,4,5,This looks good.\n\nThe code could be improved...,caram.v@turing.com
59,12/23/2023 14:40:57,youssef.h@turing.com,https://colab.research.google.com/drive/1XOkKN...,5,5,Very Good!,caram.v@turing.com
60,12/23/2023 14:47:27,youssef.h@turing.com,https://colab.research.google.com/drive/1VmX6t...,5,3,--The first assistant markdown has multiple gr...,rohit.kumar@turing.com
61,12/23/2023 14:56:30,youssef.h@turing.com,https://colab.research.google.com/drive/1CQSKk...,3,3,--Conversation does not meet the minimum lengt...,rohit.kumar@turing.com
8,12/20/2023 19:38:33,chirag.rade-c@turing.com,https://colab.research.google.com/drive/11CEko...,5,4,++ Code is clean and well structured \n++ Test...,hriday.thakkar@turing.com
9,12/20/2023 19:55:29,chirag.rade-c@turing.com,https://colab.research.google.com/drive/1dsE7l...,3,4,-- Incorrect use of f-string in the last print...,ali.nauman@turing.com
10,12/20/2023 19:57:38,chirag.rade-c@turing.com,https://colab.research.google.com/drive/1wEgUF...,5,4,++ code is clean and well structured\n++ comme...,riya.dhar@turing.com


In [26]:
from googleapiclient.discovery import build
from google.oauth2 import service_account
from datetime import datetime, timezone
import pandas as pd
from src.gdrive_api.auth import build_service
from datetime import timedelta

utc_timezone = timezone.utc
gmt_plus_2h_timezone = timezone(timedelta(hours=2))


drive_service = build_service(service_account_path)
def get_revision_before_timestamp(file_id, timestamp):
    try:
        # Get the revisions of the file
        revisions = drive_service.revisions().list(fileId=file_id).execute()
    except Exception as e:
        print(f"File not found: {file_id}")
        return None

    # Convert the timestamp to a datetime object
    
    timestamp = pd.to_datetime(timestamp).tz_localize(gmt_plus_2h_timezone)

    # Initialize the latest revision before the timestamp as None
    latest_revision_before_timestamp = None

    for revision in revisions['revisions']:
        # Convert the modifiedTime of the revision to a datetime object
        modified_time = pd.to_datetime(revision['modifiedTime'])
        # If the modifiedTime is before the timestamp
        if modified_time <= timestamp:
            # If this is the first revision or this revision is later than the latest found so far
            if latest_revision_before_timestamp is None or modified_time > pd.to_datetime(latest_revision_before_timestamp['modifiedTime']):
                # Update the latest revision before the timestamp
                latest_revision_before_timestamp = revision

    return latest_revision_before_timestamp



def get_file_id_from_task_link(task_link):
    try:
        return task_link.split("/")[-1].split('#')[0]
    except Exception as e:
        print('ERROR' + '='*60)
        print(task_link)
        return None

# Add a new column to the DataFrame for the file IDs
selected_rows_df['file_id'] = selected_rows_df['Task Link [Google Colab]'].apply(get_file_id_from_task_link)

# Apply the function to each row in the DataFrame
selected_rows_df['revision'] = selected_rows_df.apply(lambda row: get_revision_before_timestamp(row['file_id'], row['Timestamp']) if row['file_id'] is not None else None, axis=1)

2023-12-17T01:52:11.471Z
{'id': '0BxmKQQ91GHhcZE9FTzJUZnpvSkxRaG15RU1IQ1RGSkptTVNjPQ', 'mimeType': 'application/octet-stream', 'kind': 'drive#revision', 'modifiedTime': '2023-12-17T01:52:11.471Z'}
2023-12-17T02:50:08.312Z
{'id': '0BxmKQQ91GHhcNm8ya3oyUVNEQ3VlZkRQNldUQmV1b2cxVkNVPQ', 'mimeType': 'application/octet-stream', 'kind': 'drive#revision', 'modifiedTime': '2023-12-17T02:50:08.312Z'}
2023-12-18T12:57:49.487Z
{'id': '0BxmKQQ91GHhcakt4NERvc3JkRVlhOXZLblEvMUMvYnptS3RjPQ', 'mimeType': 'application/json', 'kind': 'drive#revision', 'modifiedTime': '2023-12-18T12:57:49.487Z'}
2023-12-18T14:53:41.834Z
{'id': '0BxmKQQ91GHhcR1BuUUE3cWhSUUdKZGFoWjdLM1R1NEdvK01FPQ', 'mimeType': 'application/json', 'kind': 'drive#revision', 'modifiedTime': '2023-12-18T14:53:41.834Z'}
2023-12-18T14:54:12.758Z
{'id': '0BxmKQQ91GHhcU1prWkZPSC9XblpZTlg2NUg0RUY2Y2NtMzlrPQ', 'mimeType': 'application/json', 'kind': 'drive#revision', 'modifiedTime': '2023-12-18T14:54:12.758Z'}
2023-12-18T14:57:48.638Z
{'id': '0BxmKQ

In [32]:
def append_revision_id_to_task_link(task_link, revision):
    if revision is not None:
        revision_id = revision.get('id')
        return f"{task_link}#revisionId={revision_id}"
    return task_link

selected_rows_df['task_link_rev'] = selected_rows_df.apply(lambda row: append_revision_id_to_task_link(row['Task Link [Google Colab]'], row['revision']), axis=1)


In [37]:

import sys
sys.path.insert(0, PROJECT_ROOT + '/operations/delivery/split_analytics')



from operations.delivery.split_analytics.jsonl_dump import download_and_parse_notebook

ImportError: cannot import name 'get_delivered_df' from 'utils' (/home/saya/chario/upstream_character_tasks/operations/gpt_reviews/utils.py)

In [34]:
selected_rows_df['task_link_rev']

1     https://colab.research.google.com/drive/1ZSjvo...
2     https://colab.research.google.com/drive/1nv28F...
3     https://colab.research.google.com/drive/1nBjh1...
58    https://colab.research.google.com/drive/17eg2v...
59    https://colab.research.google.com/drive/1XOkKN...
60    https://colab.research.google.com/drive/1VmX6t...
61    https://colab.research.google.com/drive/1CQSKk...
8     https://colab.research.google.com/drive/11CEko...
9     https://colab.research.google.com/drive/1dsE7l...
10    https://colab.research.google.com/drive/1wEgUF...
11    https://colab.research.google.com/drive/15AXjZ...
12    https://colab.research.google.com/drive/15e4n1...
13    https://colab.research.google.com/drive/12nfkD...
14    https://colab.research.google.com/drive/1wAM4c...
15    https://colab.research.google.com/drive/1Eopvh...
16    https://colab.research.google.com/drive/1OsBof...
17    https://colab.research.google.com/drive/1MYtCR...
18    https://colab.research.google.com/drive/14